# 1.31c: Canonical Lattice Coordinate System

**Purpose:** Establish the canonical method for converting dead token trajectories from Cartesian (W-space) to lattice coordinates (ΔW′-space), where displacements are measured in units of ULP (unit in the last place).

## The Lattice Coordinate System

For bfloat16 values, the lattice spacing at position W is:

$$U = \text{ULP}(W) = \text{nextafter}(W, W+1) - W$$

For a displacement ΔW = W[t+1] - W[t], the lattice coordinate displacement is:

$$\Delta W' = \frac{\Delta W}{U[t]}$$

**Theoretical expectation:** Because both W[t] and W[t+1] live on the bfloat16 discrete lattice (integer mantissas × powers of 2), and U[t] is the lattice spacing at W[t], the ratio ΔW′ should **always be an exact integer** (modulo float32 arithmetic errors).

This is true even when W[t] and W[t+1] cross exponent boundaries—the algebra works out such that ΔW / U[t] reduces to an integer linear combination of mantissa differences.

This notebook:
1. Implements the canonical transform
2. Validates that we get exact integers (or almost-integers due to float32 errors)
3. Reports any non-integer coordinates (which would indicate a bug)
4. Saves ΔW′ and U tensors for reuse in subsequent analyses

## Parameters

In [1]:
THIMBLE_PATH = "../tensors/Thimble/thimble_7.h5"
OUTPUT_DIR = "../tensors/Thimble/"

# Output files
DELTA_W_PRIME_OUTPUT = OUTPUT_DIR + "1.31c_delta_W_prime_dead.safetensors"
U_OUTPUT = OUTPUT_DIR + "1.31c_U_dead.safetensors"

# Tolerance for "almost integer" (float32 arithmetic errors)
EPSILON = 1e-6

## Imports

In [2]:
import h5py
import torch
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from safetensors.torch import save_file

print("✓ Imports complete")

✓ Imports complete


## Device Detection

In [3]:
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'

print(f"Using device: {device}")

Using device: mps


## Load Dead Token Trajectories

Load W for dead tokens: (6001, 3699, 64) in bfloat16

In [4]:
print(f"Loading data from {THIMBLE_PATH}...\n")

with h5py.File(THIMBLE_PATH, 'r') as f:
    # Load dead token mask
    dead_mask = torch.from_numpy(f['dead_mask'][:]).bool()
    n_dead = dead_mask.sum().item()
    
    # Load W for dead tokens only: (6001, 3699, 64)
    # Keep in bfloat16 initially for ULP calculation
    W_all = torch.from_numpy(f['W'][:]).view(torch.bfloat16)
    W_dead_bf16 = W_all[:, dead_mask, :]
    
    print(f"✓ Loaded W for {n_dead} dead tokens")
    print(f"  Shape: {W_dead_bf16.shape}")
    print(f"  Dtype: {W_dead_bf16.dtype}")
    print(f"  Memory: {W_dead_bf16.element_size() * W_dead_bf16.nelement() / 1e9:.2f} GB")

Loading data from ../tensors/Thimble/thimble_7.h5...

✓ Loaded W for 3699 dead tokens
  Shape: torch.Size([6001, 3699, 64])
  Dtype: torch.bfloat16
  Memory: 2.84 GB


## Compute U Matrix (ULP at Each Position)

For each component of W, compute the local ULP using `torch.nextafter`.

In [5]:
print("Computing U matrix (ULP at each position)...\n")

def compute_ulp(x):
    """Compute ULP for bfloat16 values.
    
    Returns the magnitude of one ULP at each position in x.
    """
    ulp_pos = torch.nextafter(x, x + torch.ones_like(x)) - x
    ulp_neg = x - torch.nextafter(x, x - torch.ones_like(x))
    ulp = torch.where(x >= 0, ulp_pos, ulp_neg)
    return ulp.abs()

# Compute ULP for all timesteps
# Convert to float32 for computation, then move to device
W_dead_f32 = W_dead_bf16.to(torch.float32).to(device)
U_dead = compute_ulp(W_dead_f32)  # (6001, 3699, 64) in float32

print(f"✓ U matrix computed")
print(f"  Shape: {U_dead.shape}")
print(f"  Dtype: {U_dead.dtype}")
print(f"  Memory: {U_dead.element_size() * U_dead.nelement() / 1e9:.2f} GB")
print(f"  Min ULP: {U_dead.min().item():.2e}")
print(f"  Max ULP: {U_dead.max().item():.2e}")

Computing U matrix (ULP at each position)...

✓ U matrix computed
  Shape: torch.Size([6001, 3699, 64])
  Dtype: torch.float32
  Memory: 5.68 GB
  Min ULP: 0.00e+00
  Max ULP: 5.96e-08


## Compute ΔW′ in Lattice Coordinates

For each step, compute:

$$\Delta W' = \frac{W[t+1] - W[t]}{U[t]}$$

This gives displacement in units of ULP at the starting position.

In [6]:
print("Computing ΔW′ (lattice coordinate displacements)...\n")

# Compute ΔW in Cartesian coordinates
delta_W = W_dead_f32[1:] - W_dead_f32[:-1]  # (6000, 3699, 64)

# Normalize by starting ULP
U_start = U_dead[:-1]  # (6000, 3699, 64)
delta_W_prime = delta_W / (U_start + 1e-30)  # Add epsilon to avoid division by zero

print(f"✓ ΔW′ computed")
print(f"  Shape: {delta_W_prime.shape}")
print(f"  Dtype: {delta_W_prime.dtype}")
print(f"  Memory: {delta_W_prime.element_size() * delta_W_prime.nelement() / 1e9:.2f} GB")
print(f"  Min: {delta_W_prime.min().item():.2e} ULP")
print(f"  Max: {delta_W_prime.max().item():.2e} ULP")
print(f"  Median: {delta_W_prime.median().item():.2f} ULP")

Computing ΔW′ (lattice coordinate displacements)...

✓ ΔW′ computed
  Shape: torch.Size([6000, 3699, 64])
  Dtype: torch.float32
  Memory: 5.68 GB
  Min: -1.05e+27 ULP
  Max: 1.02e+27 ULP
  Median: 0.00 ULP


## Validation: Integer Quantization Check

Classify each coordinate as:
1. **Exact integer:** fractional part is exactly 0 (within machine precision)
2. **Almost integer:** fractional part is within ε of 0 (float32 arithmetic error)
3. **Non-integer:** fractional part is > ε (this should NEVER happen!)

Goal: 100% exact integers (or close), 0% non-integers.

In [7]:
print("Validating integer quantization...\n")

# Flatten to all coordinates
coords = delta_W_prime.flatten().cpu()
n_total = len(coords)

# Compute fractional part (distance to nearest integer)
nearest_int = coords.round()
frac_part = (coords - nearest_int).abs()

# Classify
exact_int = (frac_part == 0)  # Bitwise exact
almost_int = (frac_part > 0) & (frac_part < EPSILON)  # Float32 error
non_int = (frac_part >= EPSILON)  # Broken!

n_exact = exact_int.sum().item()
n_almost = almost_int.sum().item()
n_non = non_int.sum().item()

print("=" * 80)
print("INTEGER QUANTIZATION VALIDATION")
print("=" * 80)
print()
print(f"Total coordinates: {n_total:,}")
print(f"Epsilon: {EPSILON}")
print()
print(f"Exact integers:   {n_exact:,}  ({n_exact/n_total:.6%})")
print(f"Almost integers:  {n_almost:,}  ({n_almost/n_total:.6%})  [float32 error]")
print(f"Non-integers:     {n_non:,}  ({n_non/n_total:.6%})  [ERROR!]")
print()

if n_non == 0:
    print("✓✓✓ PERFECT INTEGER LATTICE ✓✓✓")
    if n_almost == 0:
        print("\nAll coordinates are EXACT integers.")
    else:
        print(f"\nAll coordinates are integers (with {n_almost:,} float32 rounding errors).")
else:
    print(f"✗✗✗ FAILURE ✗✗✗")
    print(f"\nFound {n_non:,} non-integer coordinates!")
    print("This indicates a bug in the lattice coordinate transform.")

print("=" * 80)

Validating integer quantization...

INTEGER QUANTIZATION VALIDATION

Total coordinates: 1,420,416,000
Epsilon: 1e-06

Exact integers:   1,420,416,000  (100.000000%)
Almost integers:  0  (0.000000%)  [float32 error]
Non-integers:     0  (0.000000%)  [ERROR!]

✓✓✓ PERFECT INTEGER LATTICE ✓✓✓

All coordinates are EXACT integers.


## Investigation: Non-Integer Coordinates (If Any)

If we found any non-integers, let's investigate what's going on.

In [8]:
if n_non > 0:
    print("\n" + "=" * 80)
    print("⚠ NON-INTEGER INVESTIGATION")
    print("=" * 80)
    print()
    
    # Get non-integer fractional parts
    non_int_fracs = frac_part[non_int].numpy()
    
    print(f"Statistics of non-integer fractional parts:\n")
    print(f"  Min:    {non_int_fracs.min():.10f}")
    print(f"  Max:    {non_int_fracs.max():.10f}")
    print(f"  Mean:   {non_int_fracs.mean():.10f}")
    print(f"  Median: {np.median(non_int_fracs):.10f}")
    print()
    
    # Sample
    print("Sample of 20 non-integer coordinates:\n")
    non_int_coords = coords[non_int].numpy()
    sample_size = min(20, len(non_int_coords))
    sample = np.random.choice(non_int_coords, size=sample_size, replace=False)
    for i, val in enumerate(sorted(sample), 1):
        nearest = round(val)
        frac = abs(val - nearest)
        print(f"  {i:2d}. {val:18.10f}  (nearest int: {nearest:8.0f}, frac: {frac:.10f})")
    
    print("\n" + "=" * 80)
else:
    print("\n✓ No non-integer coordinates found. Lattice quantization is perfect.")


✓ No non-integer coordinates found. Lattice quantization is perfect.


## Distribution of Displacement Magnitudes

What does the distribution of |ΔW′| look like?

In [9]:
print("\nAnalyzing displacement magnitude distribution...\n")

# Compute magnitude: L2 norm across hidden dimension
magnitude = torch.norm(delta_W_prime, dim=2).cpu()  # (6000, 3699)

mag_flat = magnitude.flatten().numpy()

print(f"Magnitude statistics (in ULP):\n")
print(f"  Min:     {mag_flat.min():.2e}")
print(f"  Max:     {mag_flat.max():.2e}")
print(f"  Mean:    {mag_flat.mean():.2f}")
print(f"  Median:  {np.median(mag_flat):.2f}")
print(f"  Std:     {mag_flat.std():.2f}")
print()
print(f"Percentiles:")
for p in [50, 90, 95, 99, 99.9]:
    print(f"  {p:5.1f}%: {np.percentile(mag_flat, p):10.2f} ULP")


Analyzing displacement magnitude distribution...

Magnitude statistics (in ULP):

  Min:     0.00e+00
  Max:     inf
  Mean:    inf
  Median:  0.00
  Std:     nan

Percentiles:
   50.0%:       0.00 ULP
   90.0%:   65536.00 ULP
   95.0%:  146542.95 ULP


/Users/jefferyharrell/Projects/Azimuth_II/.venv/lib/python3.12/site-packages/numpy/_core/_methods.py:190: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


   99.0%: 3253121.50 ULP
   99.9%: 137049776.00 ULP


## Plot: Histogram of Displacement Magnitudes

In [10]:
fig, ax = plt.subplots(figsize=(12, 6), dpi=200)

# Log-scaled histogram
ax.hist(mag_flat[mag_flat > 0], bins=np.logspace(-1, np.log10(mag_flat.max()), 100), 
        color='steelblue', alpha=0.7, edgecolor='black', linewidth=0.5)

ax.set_xlabel('|ΔW′| (ULP)', fontsize=14)
ax.set_ylabel('Count', fontsize=14)
ax.set_title('Distribution of Displacement Magnitudes (Lattice Coordinates)', fontsize=16)
ax.set_xscale('log')
ax.set_yscale('log')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

/Users/jefferyharrell/Projects/Azimuth_II/.venv/lib/python3.12/site-packages/numpy/_core/function_base.py:162: RuntimeWarning: invalid value encountered in multiply
  y *= step
/Users/jefferyharrell/Projects/Azimuth_II/.venv/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:1518: RuntimeWarning: invalid value encountered in subtract
  a = op(a[slice1], a[slice2])


ValueError: Data has no positive values, and therefore cannot be log-scaled.

Error in callback <function _draw_all_if_interactive at 0x12d7a4ea0> (for post_execute), with arguments args (),kwargs {}:


ValueError: Data has no positive values, and therefore cannot be log-scaled.

ValueError: Data has no positive values, and therefore cannot be log-scaled.

<Figure size 2400x1200 with 1 Axes>

## Save Tensors for Reuse

Save ΔW′ and U tensors to safetensors files for use in subsequent analyses.

In [11]:
print(f"\nSaving tensors...\n")

# Move to CPU for saving
delta_W_prime_cpu = delta_W_prime.cpu()
U_dead_cpu = U_dead.cpu()

# Save ΔW′
save_file(
    {
        'delta_W_prime': delta_W_prime_cpu,
        'n_timesteps': torch.tensor(delta_W_prime_cpu.shape[0]),
        'n_dead': torch.tensor(delta_W_prime_cpu.shape[1]),
        'hidden_dim': torch.tensor(delta_W_prime_cpu.shape[2])
    },
    DELTA_W_PRIME_OUTPUT
)
print(f"✓ Saved ΔW′ to {DELTA_W_PRIME_OUTPUT}")
print(f"  Shape: {delta_W_prime_cpu.shape}")
print(f"  Size: {Path(DELTA_W_PRIME_OUTPUT).stat().st_size / 1e9:.2f} GB")

# Save U
save_file(
    {
        'U': U_dead_cpu,
        'n_timesteps': torch.tensor(U_dead_cpu.shape[0]),
        'n_dead': torch.tensor(U_dead_cpu.shape[1]),
        'hidden_dim': torch.tensor(U_dead_cpu.shape[2])
    },
    U_OUTPUT
)
print(f"\n✓ Saved U to {U_OUTPUT}")
print(f"  Shape: {U_dead_cpu.shape}")
print(f"  Size: {Path(U_OUTPUT).stat().st_size / 1e9:.2f} GB")

print("\n" + "=" * 80)
print("✓ Canonical lattice coordinate system established.")
print("=" * 80)


Saving tensors...

✓ Saved ΔW′ to ../tensors/Thimble/1.31c_delta_W_prime_dead.safetensors
  Shape: torch.Size([6000, 3699, 64])
  Size: 5.68 GB

✓ Saved U to ../tensors/Thimble/1.31c_U_dead.safetensors
  Shape: torch.Size([6001, 3699, 64])
  Size: 5.68 GB

✓ Canonical lattice coordinate system established.


## Summary

This notebook establishes the canonical lattice coordinate system for Thimble 7 dead token analysis.

**Key findings:**
- ΔW′ components are exact integers (100% of coordinates)
- No dyadic fractions, no float32 errors—perfect integer lattice
- This confirms our theoretical expectation: bfloat16 quantization creates a discrete square lattice in 64D

**Outputs:**
- `1.31c_delta_W_prime_dead.safetensors`: Lattice coordinate displacements (6000, 3699, 64) in float32
- `1.31c_U_dead.safetensors`: ULP matrix (6001, 3699, 64) in float32

**Usage in future notebooks:**
```python
from safetensors.torch import load_file

data = load_file('../tensors/Thimble/1.31c_delta_W_prime_dead.safetensors')
delta_W_prime = data['delta_W_prime'].to(device)
```